Import relevant packages. Uncomment html5lib and/or geocoder installation if necessary.

In [1]:
#!pip install html5lib
#!pip install geocoder
from bs4 import BeautifulSoup 
import requests
import pandas as pd
import numpy as np
import urllib.request
import folium
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

# Getting neighborhood information.
Scrape the table of Toronto neighbourhoods from Wikipedia and store it in the variable 'table'. <br>
The table I'm using has a slightly different structure from what is described in the assignment information, but this will not cause any problems other than some superficial issues with formatting in the 'Neighborhoods' column later.

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
data  = requests.get(url).text 
soup = BeautifulSoup(data, 'html5lib')
table = soup.find('table')
#table

Load 'table' as a dataframe 'df'.

In [3]:
df = pd.read_html(str(table), flavor='bs4')[0]
print(df.shape)
df.head()

(20, 9)


,0,1,2,3,4,5,6,7,8
0,M1ANot assigned,M2ANot assigned,M3ANorth York(Parkwoods),M4ANorth York(Victoria Village),M5ADowntown Toronto(Regent Park / Harbourfront),M6ANorth York(Lawrence Manor / Lawrence Heights),M7AQueen's Park(Ontario Provincial Government),M8ANot assigned,M9AEtobicoke(Islington Avenue)
1,M1BScarborough(Malvern / Rouge),M2BNot assigned,M3BNorth York(Don Mills)North,M4BEast York(Parkview Hill / Woodbine Gardens),"M5BDowntown Toronto(Garden District, Ryerson)",M6BNorth York(Glencairn),M7BNot assigned,M8BNot assigned,M9BEtobicoke(West Deane Park / Princess Garden...
2,M1CScarborough(Rouge Hill / Port Union / Highl...,M2CNot assigned,M3CNorth York(Don Mills)South(Flemingdon Park),M4CEast York(Woodbine Heights),M5CDowntown Toronto(St. James Town),M6CYork(Humewood-Cedarvale),M7CNot assigned,M8CNot assigned,M9CEtobicoke(Eringate / Bloordale Gardens / Ol...
3,M1EScarborough(Guildwood / Morningside / West ...,M2ENot assigned,M3ENot assigned,M4EEast Toronto(The Beaches),M5EDowntown Toronto(Berczy Park),M6EYork(Caledonia-Fairbanks),M7ENot assigned,M8ENot assigned,M9ENot assigned
4,M1GScarborough(Woburn),M2GNot assigned,M3GNot assigned,M4GEast York(Leaside),M5GDowntown Toronto(Central Bay Street),M6GDowntown Toronto(Christie),M7GNot assigned,M8GNot assigned,M9GNot assigned


Reorganize df into something with the correct format and roughly correct entries.

In [4]:
#First get values from df as a long list.
values = df.values.reshape(180,1)

# Now construct a 180 x 3 array containing postcode, borough and neighbourhood data in columns
data = []
for i in range(len(values)):
    text = values[i][0]
    row  = []
    row.append(text[0:3])
    text = text[3:]
    k = text.find('(')
    if k != -1:
        row.append(text[:k])
        row.append(text[k:])
    else:
        row.append(text)
        row.append(text)
    data.append(row)

# Finally rebuild dataframe using this data and appropriate column headings
columns = ['PostalCode', 'Borough', 'Neighborhood']
df = pd.DataFrame(data, columns = columns)
print(df.shape)
df.head(10)

(180, 3)


,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,(Parkwoods)
3,M4A,North York,(Victoria Village)
4,M5A,Downtown Toronto,(Regent Park / Harbourfront)
5,M6A,North York,(Lawrence Manor / Lawrence Heights)
6,M7A,Queen's Park,(Ontario Provincial Government)
7,M8A,Not assigned,Not assigned
8,M9A,Etobicoke,(Islington Avenue)
9,M1B,Scarborough,(Malvern / Rouge)


Now drop 'Not assigned' rows.

In [5]:
# First assign NaN to 'Not assigned' values in 'Borough' column
df['Borough'] = df['Borough'].replace('Not assigned', np.NaN)

# Now use dropna to drop corresponding rows.
df.dropna(inplace = True)
df.reset_index(drop = True, inplace = True)
pd.set_option('display.max_rows', None)
display(df)

,PostalCode,Borough,Neighborhood
0,M3A,North York,(Parkwoods)
1,M4A,North York,(Victoria Village)
2,M5A,Downtown Toronto,(Regent Park / Harbourfront)
3,M6A,North York,(Lawrence Manor / Lawrence Heights)
4,M7A,Queen's Park,(Ontario Provincial Government)
5,M9A,Etobicoke,(Islington Avenue)
6,M1B,Scarborough,(Malvern / Rouge)
7,M3B,North York,(Don Mills)North
8,M4B,East York,(Parkview Hill / Woodbine Gardens)
9,M5B,Downtown Toronto,"(Garden District, Ryerson)"


Now fix the formatting of entries in the 'Neighbourhood' column.

In [6]:
# First fix problems caused by words like 'South', e.g. in '(Don Mills)South(Flemingdon Park)'.
# Here I just replace things like ')South' with ' South, ', to give the correct spacing.
replacements = {'\)North' : ' North, ', '\)South' : ' South, ', '\)West' : ' West, ', '\)East' : ' East, '}
df['Neighborhood'] = df['Neighborhood'].replace(replacements, regex = True)

# Now remove remaining parenthesis and replace '/' with ','.
replacements = {'\(': '', '\)':'', ' /': ','}
df['Neighborhood'] = df['Neighborhood'].replace(replacements, regex = True)

# Finally strip trailing whitespace and commas
df['Neighborhood'] = df['Neighborhood'].apply(lambda x: x.rstrip())
df['Neighborhood'] = df['Neighborhood'].apply(lambda x: x.rstrip(','))
df


,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills North
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [7]:
df.shape

(103, 3)

# Getting geographic information.
I wrote some code (now commented out) to find coordinates using geocoder, but it doesn't work. The problem is that geocoder isn't working for, as when I try to create a geocoder object I get 'request denied'.

In [8]:
import geocoder

g = geocoder.google('M3A, Toronto, Ontario')
g

#coords = []

#for i in range(103):
#    code = df['PostalCode'].values[i]
#    lat_lng_coords = None
#    while(lat_lng_coords is None):
#          g = geocoder.google('{}, Toronto, Ontario'.format(code))
#          lat_lng_coords = g.latlng
#    coords[i] = lat_lng_coords
#    print(code + ' found')
    
#coords

<[REQUEST_DENIED] Google - Geocode [empty]>

Since geocoder is not working, I used the .csv file of coordinates provided.

In [9]:
url = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs_v1/Geospatial_Coordinates.csv'
filename = 'Geospatial_Coordinates.csv'
urllib.request.urlretrieve(url, filename)
coords = pd.read_csv('Geospatial_Coordinates.csv')
coords.rename(columns = {'Postal Code': 'PostalCode'}, inplace = True)
coords

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


It's convenient to set the indices of df and coords to being 'PostalCode'.

In [10]:
coords.set_index('PostalCode', inplace= True)
df.set_index('PostalCode', inplace = True)


Now obtain the datframe of neighborhoods and coordinates by joining df and coords. Also reset the index.

In [11]:
df_geo = pd.concat([df, coords], axis=1, join="inner")
df_geo.reset_index(inplace=True)
#df_geo['PostalCode'].apply(str)
print(df_geo.dtypes)
display(df_geo)


PostalCode       object
Borough          object
Neighborhood     object
Latitude        float64
Longitude       float64
dtype: object


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


View post codes of Toronto.

In [12]:
lat = 43.6532
long = -79.3832
map_toronto = folium.Map(location=[lat, long], zoom_start=10, control_scale = True)

for lat, lng, code in zip(df_geo['Latitude'], df_geo['Longitude'], df_geo['PostalCode']):
    label = '{}'.format(code)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto) 
    
map_toronto

# Neighborhood cluster analysis.

Now prepare Foursquare client information.

In [13]:
CLIENT_ID = '*****' 
CLIENT_SECRET = '*****' 
VERSION = '20180605' 

Now obtain information about nearby places from Foursquare. Basedon the map created earlier, I set the radius to 1300, as this is a reasonably short distance, but it's also long enough that every post code has at least one associated venue. Also, distances between neighborhood centers is fairly large, as can be seen in the map above, so a reasonably large radius is needed for good coverage.<br>
Note that this will double count some venues, as they may be within the defined search radius of more than one post code. This is not a problem, as we are interested in the venues that are easily accessible from each post code, not the total number of venues in Toronto.

In [14]:
LIMIT = 100
radius = 1300
places_jsons = []
for index, row in df_geo.iterrows():
    neighborhood_places = []
    neighborhood_latitude = row['Latitude']
    neighborhood_longitude = row['Longitude']
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION, neighborhood_latitude, neighborhood_longitude, radius, LIMIT)
    places_jsons.append(requests.get(url).json())
    

Combine the venue information collected above with neighborhood data.

In [15]:
places = []
for i in range(103):
    code = df_geo.loc[i,'PostalCode']
    borough = df_geo.loc[i,'Borough']
    neigh = df_geo.loc[i,'Neighborhood']
    neigh_lat = df_geo.loc[i,'Latitude']
    neigh_long = df_geo.loc[i,'Longitude']
    for place in places_jsons[i]["response"]['groups'][0]['items']:
        places.append([code, borough, neigh, neigh_lat, neigh_long,
                      place['venue']['name'], 
                      place['venue']['location']['lat'], 
                      place['venue']['location']['lng'],  
                      place['venue']['categories'][0]['name']])
print(len(places))
    
    

6059


Turn this into a dataframe.

In [16]:
columns = ['PostalCode', 'Borough', 'Nearby Neighborhoods', 'Area Latitude', 'Area Longitue', 'Venue Name', 
           'Venue Latitude', 'Venue Longitude', 'Venue Category']
df_places = pd.DataFrame(data = places, columns = columns)
df_places.head(10)

,PostalCode,Borough,Nearby Neighborhoods,Area Latitude,Area Longitue,Venue Name,Venue Latitude,Venue Longitude,Venue Category
0,M3A,North York,Parkwoods,43.753259,-79.329656,Allwyn's Bakery,43.759840,-79.324719,Caribbean Restaurant
1,M3A,North York,Parkwoods,43.753259,-79.329656,Bruno's valu-mart,43.746143,-79.324630,Grocery Store
2,M3A,North York,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
3,M3A,North York,Parkwoods,43.753259,-79.329656,Donalda Golf & Country Club,43.752816,-79.342741,Golf Course
4,M3A,North York,Parkwoods,43.753259,-79.329656,A&W,43.760643,-79.326865,Fast Food Restaurant
5,M3A,North York,Parkwoods,43.753259,-79.329656,Shoppers Drug Mart,43.745315,-79.325800,Pharmacy
6,M3A,North York,Parkwoods,43.753259,-79.329656,Food Basics,43.760549,-79.326045,Supermarket
7,M3A,North York,Parkwoods,43.753259,-79.329656,High Street Fish & Chips,43.745260,-79.324949,Fish & Chips Shop
8,M3A,North York,Parkwoods,43.753259,-79.329656,Pizza Pizza,43.760231,-79.325666,Pizza Place
9,M3A,North York,Parkwoods,43.753259,-79.329656,Petro-Canada,43.757950,-79.315187,Gas Station


In [17]:
df_places.groupby('PostalCode').count()

,Borough,Nearby Neighborhoods,Area Latitude,Area Longitue,Venue Name,Venue Latitude,Venue Longitude,Venue Category
PostalCode,,,,,,,,
M1B,26,26,26,26,26,26,26,26
M1C,6,6,6,6,6,6,6,6
M1E,27,27,27,27,27,27,27,27
M1G,22,22,22,22,22,22,22,22
M1H,43,43,43,43,43,43,43,43
M1J,27,27,27,27,27,27,27,27
M1K,34,34,34,34,34,34,34,34
M1L,34,34,34,34,34,34,34,34
M1M,30,30,30,30,30,30,30,30


In [18]:
print('There are {} uniques categories.'.format(len(df_places['Venue Category'].unique())))
#print(df_places['Venue Category'].unique())

There are 336 uniques categories.


In [19]:
places_onehot = pd.get_dummies(df_places['Venue Category'])
places_onehot['PostalCode'] = df_places['PostalCode'] 
fixed_columns = [places_onehot.columns[-1]] + list(places_onehot.columns[:-1])
places_onehot = places_onehot[fixed_columns]
print(places_onehot.shape)
pd.set_option('display.max_columns', None)
places_onehot.head(10)

(6059, 337)


,PostalCode,ATM,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Big Box Store,Bike Shop,Bistro,Board Shop,Bookstore,Botanical Garden,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Campground,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cemetery,Cheese Shop,Chinese Restaurant,Chocolate Shop,Churrascaria,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Quad,College Rec Center,College Stadium,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Go Kart Track,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hong Kong Restaurant,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Light Rail Station,Lighting Store,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts School,Mattress Store,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Moving Target,Museum,Music School,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Newsagent,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Outdoors & Recreation,Paintball Field,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Service,Pet Store,Pharmacy,Photography Lab,Pide Place,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Pub,Racetrack,Ramen Restaurant,Record Shop,Recording Studio,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,River,Rock Climbing Spot,Rock Club,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Science Museum,Seafood Restaurant,Shanghai Restauran

Now create a df with the total number of each venue type for each post code.

In [20]:
places_counts = places_onehot.groupby('PostalCode').sum().reset_index()
print(places_counts.shape)
places_counts.describe()

(103, 337)


,ATM,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Big Box Store,Bike Shop,Bistro,Board Shop,Bookstore,Botanical Garden,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Campground,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cemetery,Cheese Shop,Chinese Restaurant,Chocolate Shop,Churrascaria,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Quad,College Rec Center,College Stadium,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Go Kart Track,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hong Kong Restaurant,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Light Rail Station,Lighting Store,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts School,Mattress Store,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Moving Target,Museum,Music School,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Newsagent,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Outdoors & Recreation,Paintball Field,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Service,Pet Store,Pharmacy,Photography Lab,Pide Place,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Pub,Racetrack,Ramen Restaurant,Record Shop,Recording Studio,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,River,Rock Climbing Spot,Rock Club,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Science Museum,Seafood Restaurant,Shanghai Restaurant,Shoe Stor

In [21]:
places_counts.head()

,PostalCode,ATM,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Big Box Store,Bike Shop,Bistro,Board Shop,Bookstore,Botanical Garden,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Campground,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cemetery,Cheese Shop,Chinese Restaurant,Chocolate Shop,Churrascaria,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Quad,College Rec Center,College Stadium,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Go Kart Track,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hong Kong Restaurant,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Light Rail Station,Lighting Store,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts School,Mattress Store,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Moving Target,Museum,Music School,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Newsagent,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Outdoors & Recreation,Paintball Field,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Service,Pet Store,Pharmacy,Photography Lab,Pide Place,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Pub,Racetrack,Ramen Restaurant,Record Shop,Recording Studio,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,River,Rock Climbing Spot,Rock Club,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Science Museum,Seafood Restaurant,Shanghai Restauran

I will experiment by using k-means clustering on both the count data obtained above, and also on data normalized by dividing the entries in places_counts by the sum for the corresponding column. This is roughly dividing by the total number of such venues, but not quite as there is double counting as discussed above. <br>
This is not the same as the approach taken in the example notebook, as there the mean() function is used (i.e. places_onehot.groupby('PostalCode').mean().reset_index()). This effectively gives the number of times a venue type occurs in a neighborhood as a fraction of the total number of venues in the neighborhood. The problem with this is that if say a neighborhood has one Chinese restaurant, and nothing else, it gets a higher score for Chinese restaurants than a neighbourhood with 10 Chinese restaurants and one of something else. Similarly, if a neighborhood has one airport and lots of other things, then it gets a similarly low score for airports as neighborhoods with no airports at all. This seems illogical to me, so I changed the approach to normalization to the one I descibed above. <br>
Note that this alternative approach will produce a categorization that assigns most areas the same group, which I discuss discussed below. The approach with no normalization produces more evenly distributed groups. I include both approaches for comparison.  

In [22]:
data = places_counts['PostalCode']
places_normed = pd.DataFrame(data = data, columns = ['PostalCode'])
for col in places_counts:
    if col != 'PostalCode':
        places_normed[col] = (places_counts[col])/places_counts[col].sum()
display(places_normed.head())

,PostalCode,ATM,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Big Box Store,Bike Shop,Bistro,Board Shop,Bookstore,Botanical Garden,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Campground,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cemetery,Cheese Shop,Chinese Restaurant,Chocolate Shop,Churrascaria,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Quad,College Rec Center,College Stadium,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Go Kart Track,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hong Kong Restaurant,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Light Rail Station,Lighting Store,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts School,Mattress Store,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Moving Target,Museum,Music School,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Newsagent,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Outdoors & Recreation,Paintball Field,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Service,Pet Store,Pharmacy,Photography Lab,Pide Place,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Pub,Racetrack,Ramen Restaurant,Record Shop,Recording Studio,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,River,Rock Climbing Spot,Rock Club,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Science Museum,Seafood Restaurant,Shanghai Restauran

In [23]:
# set number of clusters
kclusters = 5

places_normed_clustering = places_normed.drop('PostalCode', 1)
places_counts_clustering = places_counts.drop('PostalCode', 1)


# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(places_normed_clustering)
kmeans2 = KMeans(n_clusters=kclusters, random_state=0).fit(places_counts_clustering)


# check cluster labels generated for each row in the dataframe
print(kmeans.labels_)
print(kmeans2.labels_)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 4 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0
 0 0 0 2 0 0 0 0 0 0 3 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[4 4 4 4 2 4 4 4 4 4 2 4 4 4 4 4 4 4 2 4 4 2 0 2 4 4 2 2 4 2 4 4 4 4 4 4 4
 0 0 2 0 0 0 0 4 0 0 0 0 0 0 3 3 3 3 3 3 3 3 3 3 3 0 2 0 1 1 1 4 3 3 2 2 4
 4 1 1 1 1 4 4 2 0 0 0 3 2 0 4 4 4 4 2 4 4 4 4 4 4 4 4 4 4]


In [24]:
places_with_clusters_normed = places_normed.copy()
places_with_clusters_normed['Cluster'] = kmeans.labels_
fixed_columns = [places_with_clusters_normed.columns[0]] + [places_with_clusters_normed.columns[-1]] + list(places_with_clusters_normed.columns[1:-1])
places_with_clusters_normed = places_with_clusters_normed[fixed_columns]
display(places_with_clusters_normed.head(5))

places_with_clusters = places_counts.copy()
places_with_clusters['Cluster'] = kmeans2.labels_
fixed_columns = [places_with_clusters.columns[0]] + [places_with_clusters.columns[-1]] + list(places_with_clusters.columns[1:-1])
places_with_clusters = places_with_clusters[fixed_columns]
display(places_with_clusters.head(5))

                 
                 
                 

,PostalCode,Cluster,ATM,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Big Box Store,Bike Shop,Bistro,Board Shop,Bookstore,Botanical Garden,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Campground,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cemetery,Cheese Shop,Chinese Restaurant,Chocolate Shop,Churrascaria,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Quad,College Rec Center,College Stadium,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Go Kart Track,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hong Kong Restaurant,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Light Rail Station,Lighting Store,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts School,Mattress Store,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Moving Target,Museum,Music School,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Newsagent,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Outdoors & Recreation,Paintball Field,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Service,Pet Store,Pharmacy,Photography Lab,Pide Place,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Pub,Racetrack,Ramen Restaurant,Record Shop,Recording Studio,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,River,Rock Climbing Spot,Rock Club,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Science Museum,Seafood Restaurant,Shanghai R

,PostalCode,Cluster,ATM,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Big Box Store,Bike Shop,Bistro,Board Shop,Bookstore,Botanical Garden,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Campground,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cemetery,Cheese Shop,Chinese Restaurant,Chocolate Shop,Churrascaria,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Quad,College Rec Center,College Stadium,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Go Kart Track,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hong Kong Restaurant,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Light Rail Station,Lighting Store,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts School,Mattress Store,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Moving Target,Museum,Music School,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Newsagent,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Outdoors & Recreation,Paintball Field,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Service,Pet Store,Pharmacy,Photography Lab,Pide Place,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Pub,Racetrack,Ramen Restaurant,Record Shop,Recording Studio,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,River,Rock Climbing Spot,Rock Club,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Science Museum,Seafood Restaurant,Shanghai R

The classification with normalization has almost everything in the same cluster. We should look at what is happening here. First get the post codes of the singleton clusters.

In [25]:
blob = places_with_clusters_normed['Cluster'].mode()[0]
blob

0

Get a list of occuring venues and their normed frequencies for every neighborhood not in the 'blob' cluster.

In [26]:
mask = places_with_clusters_normed['Cluster'] != blob
display(places_with_clusters_normed[mask])
for index, row in places_with_clusters_normed[mask].iterrows():
    print('Post code: ' + row['PostalCode'])
    for i in range(2,len(row)):
        if row[i] > 0:
            print(places_with_clusters.columns[i] +':' + str(row[i]))
    print('')


,PostalCode,Cluster,ATM,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Big Box Store,Bike Shop,Bistro,Board Shop,Bookstore,Botanical Garden,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Campground,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cemetery,Cheese Shop,Chinese Restaurant,Chocolate Shop,Churrascaria,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Quad,College Rec Center,College Stadium,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Go Kart Track,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hong Kong Restaurant,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Light Rail Station,Lighting Store,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts School,Mattress Store,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Moving Target,Museum,Music School,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Newsagent,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Outdoors & Recreation,Paintball Field,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Service,Pet Store,Pharmacy,Photography Lab,Pide Place,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Pub,Racetrack,Ramen Restaurant,Record Shop,Recording Studio,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,River,Rock Climbing Spot,Rock Club,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Science Museum,Seafood Restaurant,Shanghai R

Post code: M4L
Asian Restaurant:0.025
BBQ Joint:0.0967741935483871
Bakery:0.02531645569620253
Bar:0.013157894736842105
Beach:0.18181818181818182
Beer Store:0.02127659574468085
Board Shop:1.0
Breakfast Spot:0.017241379310344827
Brewery:0.08823529411764706
Burrito Place:0.08333333333333333
Bus Stop:0.07692307692307693
Butcher:0.1
Café:0.016666666666666666
Coffee Shop:0.012448132780082987
Comic Shop:0.1111111111111111
Diner:0.021739130434782608
Discount Store:0.05
Donut Shop:0.14285714285714285
Dry Cleaner:0.5
Egyptian Restaurant:0.5
Farmers Market:0.030303030303030304
Fast Food Restaurant:0.03125
Food:0.3333333333333333
French Restaurant:0.041666666666666664
Furniture / Home Store:0.023809523809523808
Garden Center:0.3333333333333333
Gas Station:0.015151515151515152
Grocery Store:0.015748031496062992
Gym / Fitness Center:0.045454545454545456
Harbor / Marina:0.23076923076923078
Indian Chinese Restaurant:0.3333333333333333
Indian Restaurant:0.109375
Indie Theater:0.5
Italian Restaurant:0.0

Notice that all the postcodes in non-modal clusters contain the only occurrence of some venue types. This is skewing the analysis, as most normalized venue frequencies are small, and having value 1 means every other area has value zero for any given venue. So any venue with a value of one for a given area makes a significant contribution to the classification of that area.  Also, many of these unique labels are things like 'Mac and Cheese Joint', which should really be included under the heading 'Burger joint'. 

Now combine cluster information with geographic information. 

In [27]:
places_with_clusters.set_index('PostalCode', inplace = True)
places_with_clusters_normed.set_index('PostalCode', inplace = True)
df_geo2 = df_geo.set_index('PostalCode')
geo_clusters = pd.concat([df_geo2, places_with_clusters_normed], axis=1, join="inner")
geo_clusters2 = pd.concat([df_geo2, places_with_clusters], axis=1, join="inner")
geo_clusters.reset_index(inplace = True)
geo_clusters2.reset_index(inplace = True)
#display(geo_clusters.head())
#display(geo_clusters2.head())

Here is the map of area classifications using the normed data.

In [28]:
lat = 43.6532
long = -79.3832
map_toronto_clusters = folium.Map(location=[lat, long], zoom_start=10, control_scale = True)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

for lat, lng, code, cluster in zip(geo_clusters['Latitude'], geo_clusters['Longitude'], geo_clusters['PostalCode'], geo_clusters['Cluster']):
    label = 'Code: {}, Cluster: {}'.format(code,cluster)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto_clusters) 
    
map_toronto_clusters



Now create a map of neighborhood clusters created without normalizing.

In [29]:
lat = 43.6532
long = -79.3832
map_toronto_clusters2 = folium.Map(location=[lat, long], zoom_start=10, control_scale = True)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

for lat, lng, code, cluster in zip(geo_clusters2['Latitude'], geo_clusters2['Longitude'], geo_clusters2['PostalCode'], geo_clusters2['Cluster']):
    label = 'Code: {}, Cluster: {}'.format(code,cluster)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto_clusters2) 
    
map_toronto_clusters2


For each cluster obtained using the unnormed data, I will try to profile the kind of areas it contains. I will do this by taking the average of each venue type across neighborhoods in the cluster, and subtracting from it the average fro that venue type across all neighborhoods. This will give for each venue type a value indicating how common that venue type is relative to the 'global average'. The idea is that by looking at unusually common and unusually uncommon venue types, we can get a feel for what the neighborhoods are like.

In [30]:
places_with_clusters.reset_index(inplace = True)
mask_0 = places_with_clusters['Cluster'] == 0
cluster_0 = places_with_clusters[mask_0]
cluster_0 = cluster_0.drop(['PostalCode', 'Cluster'], axis = 1)
venues_0 = pd.DataFrame(data = cluster_0.columns, columns = ['Venue type'])
venues_0.set_index('Venue type', inplace = True)
means_0 = pd.DataFrame(data = cluster_0.mean(), columns = ['Average count'])
cluster_0 = pd.concat([venues_0, means_0], axis=1, join="inner")
for index, row in cluster_0.iterrows():
    cluster_0.at[index,'Average count'] = cluster_0.loc[index,'Average count']-places_counts[index].mean()
cluster_0 = cluster_0.sort_values('Average count', ascending = False)
display(cluster_0.head(10))
display(cluster_0.tail(10))


,Average count
Coffee Shop,2.899336
Italian Restaurant,1.927440
Sushi Restaurant,1.530404
Café,1.406745
Bank,1.383240
Bakery,1.308125
Pub,1.287174
Pizza Place,1.052632
Bar,1.051610
Indian Restaurant,0.957588


,Average count
Shopping Mall,-0.162494
Chinese Restaurant,-0.171180
Hotel,-0.194686
Furniture / Home Store,-0.197241
Art Gallery,-0.199796
Beer Bar,-0.205416
Theater,-0.209504
Cocktail Bar,-0.248339
Intersection,-0.288707
Cosmetics Shop,-0.310680


Cluster 0 has a fairly high frequency of some kinds of restaurants and bars. The low frequency of other kinds of bars indicates that we could likely benefit from merging several of the venue types (though I'm not going to do that here).

In [31]:
mask_1 = places_with_clusters['Cluster'] == 1
cluster_1 = places_with_clusters[mask_1]
cluster_1 = cluster_1.drop(['PostalCode', 'Cluster'], axis = 1)
venues_1 = pd.DataFrame(data = cluster_1.columns, columns = ['Venue type'])
venues_1.set_index('Venue type', inplace = True)
means_1 = pd.DataFrame(data = cluster_1.mean(), columns = ['Average count'])
cluster_1 = pd.concat([venues_1, means_1], axis=1, join="inner")
for index, row in cluster_1.iterrows():
    cluster_1.at[index,'Average count'] = cluster_1.loc[index,'Average count']-places_counts[index].mean()
cluster_1 = cluster_1.sort_values('Average count', ascending = False)
display(cluster_1.head(10))
display(cluster_1.tail(10))

,Average count
Café,5.812760
Bar,2.976422
Bakery,2.751734
Vegetarian / Vegan Restaurant,2.342580
Italian Restaurant,2.183079
Cocktail Bar,1.556172
Restaurant,1.291262
Beer Bar,1.260749
Grocery Store,1.195562
Dessert Shop,1.095700


,Average count
Hotel,-0.277393
Sporting Goods Shop,-0.300971
Beer Store,-0.313454
Skating Rink,-0.359223
Bank,-0.443828
Intersection,-0.446602
Chinese Restaurant,-0.449376
Pharmacy,-0.464632
Gas Station,-0.640777
Fast Food Restaurant,-0.789182


Cluster 1 has very high numbers of cafes and bars, and some kinds of restaurants. It seems consistent with the 'downtown' location suggested by the map.

In [32]:
mask_2 = places_with_clusters['Cluster'] == 2
cluster_2 = places_with_clusters[mask_2]
cluster_2 = cluster_2.drop(['PostalCode', 'Cluster'], axis = 1)
venues_2 = pd.DataFrame(data = cluster_2.columns, columns = ['Venue type'])
venues_2.set_index('Venue type', inplace = True)
means_2 = pd.DataFrame(data = cluster_2.mean(), columns = ['Average count'])
cluster_2 = pd.concat([venues_2, means_2], axis=1, join="inner")
for index, row in cluster_2.iterrows():
    cluster_2.at[index,'Average count'] = cluster_2.loc[index,'Average count']-places_counts[index].mean()
cluster_2 = cluster_2.sort_values('Average count', ascending = False)
display(cluster_2.head(10))
display(cluster_2.tail(10))

,Average count
Coffee Shop,1.120388
Clothing Store,0.895146
Gas Station,0.825890
Fast Food Restaurant,0.734628
Restaurant,0.691262
Middle Eastern Restaurant,0.550809
Burger Joint,0.513269
Fried Chicken Joint,0.425243
Sandwich Place,0.401942
Korean Restaurant,0.375405


,Average count
Thai Restaurant,-0.326861
Gastropub,-0.389644
Bar,-0.471197
Vegetarian / Vegan Restaurant,-0.514563
Italian Restaurant,-0.588350
Pub,-0.593528
Pizza Place,-0.600000
Bakery,-0.600647
Park,-0.836893
Café,-1.596764


Cluster 2 seems to have more stores and fast food restaurants. This seems consistent with the more residential, dispersed character suggested by the map.

In [33]:
mask_3 = places_with_clusters['Cluster'] == 3
cluster_3 = places_with_clusters[mask_3]
cluster_3 = cluster_3.drop(['PostalCode', 'Cluster'], axis = 1)
venues_3 = pd.DataFrame(data = cluster_3.columns, columns = ['Venue type'])
venues_3.set_index('Venue type', inplace = True)
means_3 = pd.DataFrame(data = cluster_3.mean(), columns = ['Average count'])
cluster_3 = pd.concat([venues_3, means_3], axis=1, join="inner")
for index, row in cluster_3.iterrows():
    cluster_3.at[index,'Average count'] = cluster_3.loc[index,'Average count']-places_counts[index].mean()
cluster_3 = cluster_3.sort_values('Average count', ascending = False)
display(cluster_3.head(10))
display(cluster_3.tail(10))

,Average count
Coffee Shop,3.534674
Café,3.241331
Japanese Restaurant,2.958391
Restaurant,2.576976
Hotel,1.865465
Farmers Market,1.465326
Gastropub,1.329404
Gym,1.281553
Theater,1.166436
Sushi Restaurant,1.165742


,Average count
Chinese Restaurant,-0.306519
Sandwich Place,-0.326630
Burger Joint,-0.339112
Convenience Store,-0.388350
Intersection,-0.446602
Fast Food Restaurant,-0.574896
Grocery Store,-0.590153
Gas Station,-0.640777
Pharmacy,-0.821775
Bank,-1.015257


Cluster 3 is possibly characterized by the presence of hotels and theatres, along with the usual variety of restaurants. I notice on the map this cluster is very tight and near the waterfront.

In [34]:
mask_4 = places_with_clusters['Cluster'] == 4
cluster_4 = places_with_clusters[mask_4]
cluster_4 = cluster_4.drop(['PostalCode', 'Cluster'], axis = 1)
venues_4 = pd.DataFrame(data = cluster_4.columns, columns = ['Venue type'])
venues_4.set_index('Venue type', inplace = True)
means_4 = pd.DataFrame(data = cluster_4.mean(), columns = ['Average count'])
cluster_4 = pd.concat([venues_4, means_4], axis=1, join="inner")
for index, row in cluster_4.iterrows():
    cluster_4.at[index,'Average count'] = cluster_4.loc[index,'Average count']-places_counts[index].mean()
cluster_4 = cluster_4.sort_values('Average count', ascending = False)
display(cluster_4.head(10))
display(cluster_4.tail(10))

,Average count
Intersection,0.303398
Chinese Restaurant,0.282767
Discount Store,0.139159
Pharmacy,0.127629
Shopping Mall,0.117112
Convenience Store,0.111650
Bus Stop,0.082120
Bus Line,0.070995
Gas Station,0.067557
Golf Course,0.058455


,Average count
Gym,-0.489280
Thai Restaurant,-0.514361
Bar,-0.633697
Sushi Restaurant,-0.840210
Bakery,-0.992314
Japanese Restaurant,-1.059466
Italian Restaurant,-1.075850
Restaurant,-1.167071
Café,-1.850930
Coffee Shop,-2.492112


Cluster 4 is distinctive as being characterized by utilities and public transport rather than restaurants. This is what we might expect by looking at the map.

### Summary 
Overall this cluster analysis needs some work, as clusters 0,1,3 are quite similar according to superficial analysis of venues. They are, however, distinct according to the map, so something interesting is happening, though it's not clear to me exactly what the clustering algorithm is noticing. For a better anlysis we should probably combine or delete some venue types, and we would also benefit from getting some extra socioeconomic data.